In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, ArrayType
from pyspark.sql.functions import col                       # Filtering using the col() function
from pyspark.sql.functions import array_contains            # Filtering on array columns
from pyspark.sql.functions import explode                   # Explode Arrays in Individual Rows
from pyspark.sql.functions import sum, avg, count, max      # Multiple Aggregations
from pyspark.sql.functions import first, last

# Create an entry point to the PySpark Application
spark = SparkSession.builder \
      .master("local") \
      .appName("MyFirstSparkApplication") \
      .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/09 23:34:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#IMPORT

df_book = spark.read.option("multiline","true")  \
      .json("dataset/book-db.json")

df_article = spark.read.option("multiline","true")  \
      .json("dataset/article-db.json")

df_incollection = spark.read.option("multiline","true")  \
      .json("dataset/incollection-db.json")

df_www = spark.read.option("multiline","true")  \
      .json("dataset/www-db.json")

In [3]:
df_www.printSchema()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- key: string (nullable = true)
 |-- note: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- url: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [4]:
df_book.printSchema()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- citations: long (nullable = true)
 |-- crossref: string (nullable = true)
 |-- editor: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- ee: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- isbn: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- key: string (nullable = true)
 |-- keyword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pages: long (nullable = true)
 |-- publisher: string (nullable = true)
 |-- series: struct (nullable = true)
 |    |-- href: string (nullable = true)
 |    |-- title: string (nullable = true)
 |-- title: string (nullable = true)
 |-- volume: long (nullable = true)
 |-- 

In [13]:
df_article.printSchema()
df_article.show()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- citations: long (nullable = true)
 |-- cite: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ee: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- journal: string (nullable = true)
 |-- key: string (nullable = true)
 |-- keyword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- note: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pages: long (nullable = true)
 |-- publisher: string (nullable = true)
 |-- title: string (nullable = true)
 |-- volume: long (nullable = true)
 |-- year: long (nullable = true)

+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+----+-----+---------+--------------------+------+-

In [10]:
df_incollection.printSchema()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- citations: long (nullable = true)
 |-- crossref: string (nullable = true)
 |-- ee: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- key: string (nullable = true)
 |-- keyword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pages: long (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



<h1>1.Query where e join</h1>

In [6]:
#Find articles cited in the articles with most number of citations
df_article.select(df_article.key,df_article.citations,explode(df_article.cite).alias("explcite")).\
    sort(df_article.citations.desc()).\
    join(df_article.alias("articles2"),col("articles2.key") == col("explcite"),"inner").\
    show()

+--------------------+---------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+----+-----+---------+--------------------+------+----+
|                 key|citations|            explcite|              author|citations|                cite|                  ee|             journal|                 key|             keyword|note|pages|publisher|               title|volume|year|
+--------------------+---------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+----+-----+---------+--------------------+------+----+
|journals/tkde/Del...|      141|journals/debu/Rou...|[{Nick Roussopoul...|     null|[conf/sigmod/Blak...|[http://sites.com...|IEEE Data Eng. Bull.|journals/debu/Rou...|[debugging, graph...|null|   27|     null|The ADMS Project:...|    18|1995|
|journals/vldb/Har...|  

<h1>2.Limit,like,nested and where</h1>

In [7]:
#Trova gli autori con libri con una certa keyword 
#df_book.filter(array_contains(col("keyword"), "data mining")).select(col("author")).limit(5).show(truncate = False)
explauthors = df_book.select(explode(col("author")).alias("author"),col("keyword"))
explauthors.select(col("author.name"),explode(explauthors.keyword).alias("keyword")).\
    filter(col("keyword").like("%data%") & col("author.name").isNotNull()).\
    distinct().\
    limit(5).\
    show(truncate = False)

+----------------------+-------------------+
|name                  |keyword            |
+----------------------+-------------------+
|Ligang Wu             |graph database     |
|Muhammad Shafique 0001|data mining        |
|Krishnendu Chakrabarty|data processing    |
|Georg Lauhoff         |key-value databases|
|Shakti Sahi           |database design    |
+----------------------+-------------------+



<h1>3.Group By, nested query, filter(where)</h1>
trova per ogni anno le homepage dell' autore con più articoli in quell'anno per gli anni dopo il 1980

In [8]:
articles_per_year_per_author = df_article.\
    groupBy(col("author.name").alias("author"),col("year").alias("year")).\
    count()
articles_per_year_per_author.\
    groupBy(col("year")).\
    agg(first("author").alias("author"),max("count").alias("count")).\
    filter(col("year")>1980).\
    sort(col("year").desc()).\
    show()


+----+--------------------+-----+
|year|              author|count|
+----+--------------------+-----+
|2012|[Jacques Wainer, ...|    1|
|2001| [Marianne Winslett]|    3|
|2000| [Yuqing Melanie Wu]|    6|
|1999|    [H. V. Jagadish]|    6|
|1998|[Jonathan E. Cook...|    2|
|1997|[Michael J. Carey...|    2|
|1996|[Michael Stonebra...|    2|
|1995|[Xiao-Jia M. Zhou...|    2|
|1994|[Stamatis Vassili...|    2|
|1993|[Philip S. Yu, Do...|    2|
|1992|[Vijay Kumar 0002...|    3|
|1991|[Gio Wiederhold, ...|    2|
|1990|[Sushil Jajodia, ...|    5|
|1989|[Lothar F. Macker...|    3|
|1988|[Karl-Hans Blasiu...|    3|
|1987|[Toshihide Ibarak...|    3|
|1986|       [Rudi Studer]|    1|
|1985|[Y. C. Tay, Natha...|    1|
|1984|[Wolfgang Effelsb...|    1|
|1983|  [Wiebren de Jonge]|    2|
+----+--------------------+-----+
only showing top 20 rows



<h1>4. Update row</h1>

In [69]:
#Update null "citations" fields with the length of the "cite" array field
from pyspark.sql.functions import when,size
df_article.withColumn("citations", when(df_article.citations.isNull(),size(col("cite"))).\
    otherwise(df_article.citations)).\
    select("citations").show()


+---------+
|citations|
+---------+
|       81|
|       34|
|       13|
|        5|
|       88|
|      123|
|       72|
|        0|
|        4|
|        9|
|      128|
|       12|
|       27|
|      114|
|       18|
|        8|
|      119|
|        8|
|       85|
|       36|
+---------+
only showing top 20 rows

